# Practice Notebook: Files + PostgreSQL (Codespaces)

This notebook is intentionally **repo-name independent**.

**Important:**  
- Your **GitHub repo name** (e.g., `Data_Science-notebooks`) is *not* your PostgreSQL schema.  
- Your PostgreSQL **database/schema** for this course stays `hi5304` unless your instructor explicitly changes it.

We will:
1. Load a CSV from the repo `data/` folder  
2. Connect to PostgreSQL using a single, reusable connection  
3. Run SQL queries and analyze the results with pandas


## 0) Setup (run once)

This cell:
- finds the repo root (where the `data/` folder lives)
- sets `DATA_DIR` for file loading
- creates a PostgreSQL `engine` for `pd.read_sql(...)`


In [2]:
# Load the patients CSV from the repo's data/ folder
patients = pd.read_csv(DATA_DIR / "patients.csv")

patients.head()


,patient_id,first_name,last_name,date_of_birth,sex,race,ethnicity,zip_code
0,1001,John,Doe,4/12/1968,M,White,Non-Hispanic,75201
1,1002,Maria,Lopez,9/30/1975,F,Hispanic,Hispanic,75204
2,1003,James,Smith,1/18/1982,M,Black,Non-Hispanic,75080
3,1004,Linda,Chen,6/5/1990,F,Asian,Non-Hispanic,75024
4,1005,Robert,Johnson,11/22/1959,M,White,Non-Hispanic,75230


In [3]:
from pathlib import Path
import pandas as pd

DATA_DIR = Path.cwd() / "data"
print("Working directory:", Path.cwd())
print("Data directory exists:", DATA_DIR.exists())


Working directory: /workspaces/Data_Science-notebooks/lessons
Data directory exists: False


In [ ]:
schema = SCHEMA
query = f"""
SELECT *
FROM {schema}.patients
LIMIT 5;
"""

df_patients = pd.read_sql(query, engine)
df_patients


In [ ]:
# Shape of the dataset
patients.shape


In [ ]:
# Column names
patients.columns


In [ ]:
# Basic info
patients.info()


In [ ]:
from pathlib import Path
import pandas as pd

DATA_DIR = Path("..") / "data"
patients = pd.read_csv(DATA_DIR / "patients.csv")

patients.head()


In [ ]:
schema = SCHEMA
query = f"""
SELECT *
FROM {schema}.patients
LIMIT 5;
"""

df_patients = run_sql(query)
df_patients


In [ ]:
schema = SCHEMA
query = f"""
SELECT
    p.patient_id,
    p.first_name,
    p.last_name,
    b.reading_date,
    b.systolic,
    b.diastolic,
    b.heart_rate
FROM {schema}.patients p
JOIN hi5304.bp_readings b
  ON p.patient_id = b.patient_id
ORDER BY b.reading_date;
"""

df_bp = run_sql(query)
df_bp.head()


In [ ]:
schema = SCHEMA
query = f"""
SELECT
    patient_id,
    COUNT(*) AS medication_count
FROM {schema}.medications
GROUP BY patient_id
ORDER BY medication_count DESC;
"""

df_meds = run_sql(query)
df_meds


In [ ]:
schema = SCHEMA
query = f"""
SELECT
    AVG(systolic) AS avg_systolic,
    AVG(diastolic) AS avg_diastolic,
    COUNT(*) AS total_readings
FROM {schema}.bp_readings;
"""

df_summary = run_sql(query)
df_summary


In [ ]:
df_bp.describe()


In [ ]:
df_bp.groupby("patient_id")[["systolic", "diastolic"]].mean()


In [ ]:
import matplotlib.pyplot as plt

df_bp.plot(x="reading_date", y="systolic", kind="line")
plt.show()


In [ ]:
schema = SCHEMA
query = f"""
SELECT * 
FROM {schema}.cardio1
WHERE sbp IS NOT NULL
  AND cardio IS NOT NULL;
"""
df = run_sql(query)

df.head()


In [ ]:
df["cardio"].value_counts()


In [ ]:
sbp_cardio_0.describe(), sbp_cardio_1.describe()


In [ ]:
from scipy.stats import f_oneway


In [ ]:
f_stat, p_value = f_oneway(sbp_cardio_0, sbp_cardio_1)

f_stat, p_value


In [ ]:
import matplotlib.pyplot as plt

df.boxplot(column="sbp", by="cardio")
plt.xlabel("Cardio (0 = No CVD, 1 = CVD)")
plt.ylabel("Systolic Blood Pressure (mmHg)")
plt.title("SBP by Cardiovascular Disease Status")
plt.suptitle("")
plt.show()


In [ ]:
sbp_cardio_0 = df.loc[df["cardio"] == 0, "sbp"]
sbp_cardio_1 = df.loc[df["cardio"] == 1, "sbp"]


In [ ]:
from scipy.stats import f_oneway

f_stat, p_value = f_oneway(sbp_cardio_0, sbp_cardio_1)
f_stat, p_value
